In [1]:
import tvm
from tvm import te
import tvm.relay as relay
from tvm import rpc
from tvm.contrib import utils, graph_executor as runtime
from tvm.contrib.download import download_testdata

In [2]:
from mxnet.gluon.model_zoo.vision import get_model
from PIL import Image
import numpy as np

# one line to get the model
block = get_model("resnet18_v1", pretrained=True)

In [3]:
img_url = "https://github.com/dmlc/mxnet.js/blob/main/data/cat.png?raw=true"
img_name = "cat.png"
img_path = download_testdata(img_url, img_name, module="data")
image = Image.open(img_path).resize((224, 224))


def transform_image(image):
    image = np.array(image) - np.array([123.0, 117.0, 104.0])
    image /= np.array([58.395, 57.12, 57.375])
    image = image.transpose((2, 0, 1))
    image = image[np.newaxis, :]
    return image


x = transform_image(image)

In [4]:
synset_url = "".join(
    [
        "https://gist.githubusercontent.com/zhreshold/",
        "4d0b62f3d01426887599d4f7ede23ee5/raw/",
        "596b27d23537e5a1b5751d2b0481ef172f58b539/",
        "imagenet1000_clsid_to_human.txt",
    ]
)
synset_name = "imagenet1000_clsid_to_human.txt"
synset_path = download_testdata(synset_url, synset_name, module="data")
with open(synset_path) as f:
    synset = eval(f.read())

In [5]:
# We support MXNet static graph(symbol) and HybridBlock in mxnet.gluon
shape_dict = {"data": x.shape}
mod, params = relay.frontend.from_mxnet(block, shape_dict)
# we want a probability so add a softmax operator
func = mod["main"]
func = relay.Function(func.params, relay.nn.softmax(func.body), None, func.type_params, func.attrs)

In [6]:
GPU = False
# obtain an RPC session from remote device.
# The following is my environment, change this to the IP address of your target device
host = "192.168.55.1"
port = 9090
remote = rpc.connect(host, port)


if GPU:
    target = tvm.target.Target(target='cuda -arch=sm_62', host='llvm -mtriple=aarch64-linux-gnu')
else:
    target = tvm.target.Target('llvm -mtriple=aarch64-linux-gnu -device=arm_cpu -mattr=+neon')
    # target = tvm.target.arm_cpu("rasp3b")
    # The above line is a simple form of
    # target = tvm.target.Target('llvm -device=arm_cpu -model=bcm2837 -mtriple=armv7l-linux-gnueabihf -mattr=+neon')

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(func, target, params=params)

# After `relay.build`, you will get three return values: graph,
# library and the new parameter, since we do some optimization that will
# change the parameters but keep the result of model as the same.

# Save the library at local temporary directory.
tmp = utils.tempdir()
lib_fname = tmp.relpath("net_cpu.tar")
lib.export_library(lib_fname)

dense is not optimized for arm cpu.
pool is not optimized for arm cpu.
dense is not optimized for arm cpu.


In [7]:
# upload the library to remote device and load it
remote.upload(lib_fname)
rlib = remote.load_module("net_cpu.tar")

# create the remote runtime module
dev = remote.cuda(0) if GPU else remote.cpu(0)
module = runtime.GraphModule(rlib["default"](dev))
# set input data
module.set_input("data", tvm.nd.array(x.astype("float32")))
# run
module.run()
# get output
out = module.get_output(0)
# get top1 result
top1 = np.argmax(out.numpy())
print("TVM prediction top-1: {}".format(synset[top1]))

TVM prediction top-1: tiger cat


In [8]:
ftimer = module.module.time_evaluator('run', dev)

In [9]:
np.array(ftimer().results * 1000).mean()

0.08995229369999996

CPU: 0.114ms 
GPU: 0.021ms